### 출처: Won Joon Yoo, Introduction to Deep Learning for Natural Language Processing, Wikidocs</br>
### https://wikidocs.net/book/2155

### RNN을 이용하여 텍스트 생성하기

In [2]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [4]:
import numpy as np

In [5]:
from tensorflow.keras.utils import to_categorical # 원-핫 인코딩

In [6]:
text="""경마장에 있는 말이 뛰고 있다\n
그의 말이 법이다\n
가는 말이 고와야 오는 말이 곱다\n"""

단어 집합 생성 및 크기 확인

In [7]:
t = Tokenizer()
t.fit_on_texts([text]) # str형태를 list형태로 변환

In [8]:
# 인덱스 확인
t.word_index

{'말이': 1,
 '경마장에': 2,
 '있는': 3,
 '뛰고': 4,
 '있다': 5,
 '그의': 6,
 '법이다': 7,
 '가는': 8,
 '고와야': 9,
 '오는': 10,
 '곱다': 11}

In [9]:
# 빈도 확인
t.word_counts

OrderedDict([('경마장에', 1),
             ('있는', 1),
             ('말이', 4),
             ('뛰고', 1),
             ('있다', 1),
             ('그의', 1),
             ('법이다', 1),
             ('가는', 1),
             ('고와야', 1),
             ('오는', 1),
             ('곱다', 1)])

In [10]:
vocab_size = len(t.word_index) + 1 # 케라스 원-핫 인코딩 시 배열의 인덱스가 0부터 시작하므로

In [11]:
# 단어 집합의 크기
vocab_size

12

In [12]:
sequences = list()
for line in text.split('\n'): # \n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line]) # str을 list로 변경. 숫자로 변경
    # 인덱스 [0]의 의미: [[]] 리스트 안의 리스트 구조이므로 0번째 것을 선택해야함
    print(encoded)

[[2, 3, 1, 4, 5]]
[[]]
[[6, 1, 7]]
[[]]
[[8, 1, 9, 10, 1, 11]]
[[]]


In [13]:
sequences = list()
for line in text.split('\n'): # \n을 기준으로 문장 토큰화
    encoded = t.texts_to_sequences([line])[0] # str을 list로 변경. 숫자로 변경
    # 인덱스 [0]의 의미: [[]] 리스트 안의 리스트 구조이므로 0번째 것을 선택해야함
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

In [14]:
len(sequences)

11

In [15]:
sequences

[[2, 3],
 [2, 3, 1],
 [2, 3, 1, 4],
 [2, 3, 1, 4, 5],
 [6, 1],
 [6, 1, 7],
 [8, 1],
 [8, 1, 9],
 [8, 1, 9, 10],
 [8, 1, 9, 10, 1],
 [8, 1, 9, 10, 1, 11]]

In [16]:
max_len = max(len(l) for l in sequences) # 모든 샘플 길이 중 가장 긴 것만 출력
max_len

6

In [17]:
# 마지막 열이 답(y, 레이블)이므로 padding을 반드시 pre로 해야 함
sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')

In [18]:
sequences

array([[ 0,  0,  0,  0,  2,  3],
       [ 0,  0,  0,  2,  3,  1],
       [ 0,  0,  2,  3,  1,  4],
       [ 0,  2,  3,  1,  4,  5],
       [ 0,  0,  0,  0,  6,  1],
       [ 0,  0,  0,  6,  1,  7],
       [ 0,  0,  0,  0,  8,  1],
       [ 0,  0,  0,  8,  1,  9],
       [ 0,  0,  8,  1,  9, 10],
       [ 0,  8,  1,  9, 10,  1],
       [ 8,  1,  9, 10,  1, 11]], dtype=int32)

In [19]:
# 마지막 단어를 레이블로 분리
# numpy를 이용해서
sequences = np.array(sequences)
X = sequences[:, :-1] # feature
y = sequences[:, -1] # label

In [20]:
X

array([[ 0,  0,  0,  0,  2],
       [ 0,  0,  0,  2,  3],
       [ 0,  0,  2,  3,  1],
       [ 0,  2,  3,  1,  4],
       [ 0,  0,  0,  0,  6],
       [ 0,  0,  0,  6,  1],
       [ 0,  0,  0,  0,  8],
       [ 0,  0,  0,  8,  1],
       [ 0,  0,  8,  1,  9],
       [ 0,  8,  1,  9, 10],
       [ 8,  1,  9, 10,  1]], dtype=int32)

In [21]:
y

array([ 3,  1,  4,  5,  1,  7,  1,  9, 10,  1, 11], dtype=int32)

In [22]:
# 레이블에 대해서 원-핫 인코딩 수행
# X는 임베딩 때 숫자화 될 예정
y = to_categorical(y, num_classes=vocab_size)

In [23]:
y

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

### RNN 모델에 데이터 훈련

In [24]:
from tensorflow.keras.models import Sequential

In [25]:
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN

In [27]:
model = Sequential()
# 단어 벡터 차원 10
model.add(Embedding(vocab_size, 10, input_length=max_len-1)) 
# 레이블을 분리하였으므로 이제 x의 길이는 5
model.add(SimpleRNN(32))
model.add(Dense(vocab_size, activation='softmax'))# 다중분류 softmax
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)
# verbose=2는 

Epoch 1/200
1/1 - 5s - loss: 2.4851 - accuracy: 0.0909
Epoch 2/200
1/1 - 0s - loss: 2.4701 - accuracy: 0.1818
Epoch 3/200
1/1 - 0s - loss: 2.4552 - accuracy: 0.1818
Epoch 4/200
1/1 - 0s - loss: 2.4404 - accuracy: 0.4545
Epoch 5/200
1/1 - 0s - loss: 2.4255 - accuracy: 0.4545
Epoch 6/200
1/1 - 0s - loss: 2.4104 - accuracy: 0.5455
Epoch 7/200
1/1 - 0s - loss: 2.3951 - accuracy: 0.4545
Epoch 8/200
1/1 - 0s - loss: 2.3793 - accuracy: 0.5455
Epoch 9/200
1/1 - 0s - loss: 2.3630 - accuracy: 0.5455
Epoch 10/200
1/1 - 0s - loss: 2.3462 - accuracy: 0.5455
Epoch 11/200
1/1 - 0s - loss: 2.3287 - accuracy: 0.5455
Epoch 12/200
1/1 - 0s - loss: 2.3105 - accuracy: 0.5455
Epoch 13/200
1/1 - 0s - loss: 2.2915 - accuracy: 0.5455
Epoch 14/200
1/1 - 0s - loss: 2.2718 - accuracy: 0.5455
Epoch 15/200
1/1 - 0s - loss: 2.2512 - accuracy: 0.4545
Epoch 16/200
1/1 - 0s - loss: 2.2299 - accuracy: 0.4545
Epoch 17/200
1/1 - 0s - loss: 2.2078 - accuracy: 0.4545
Epoch 18/200
1/1 - 0s - loss: 2.1850 - accuracy: 0.4545
E

모델이 정확하게 예측하는지 문장을 생성하는 함수

In [28]:
def sentence_generation(model, t, current_word, n):
    # 모델, 토크나이저, 현재 단어, 반복 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복. 인덱스 사용 안 할 경우. _(언더바) 사용
        
        # 단어를 숫자로
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=5, padding='pre') # 데이터에 대한 패딩
        # maxlen : y를 뗀 후 X의 길이값
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 Y를 예측하고 Y(예측한 단어)를 result에 저장.
        
        # 숫자를 단어로
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [29]:
print(sentence_generation(model, t, '경마장에', 4))

/home/leok/anaconda3/envs/py38/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


경마장에 있는 말이 뛰고 있다


In [30]:
# 원 문장의 개수보다 많은 숫자를 입력하면 잘못된 결과 나옴을 확인
print(sentence_generation(model, t, '경마장에', 5))

경마장에 있는 말이 뛰고 있다 뛰고
